# The battle between "Bronx" and "Queens"

## Introduction
In this lap the stepwise approach is described how to get the data needed for the comparision between the boroughs of "Bronx" and "Queens", how to mind the data and how Foursquare API is used to get the data for furhter neighborhood data for the market analysis by k-means cluster. 


### Import the required libraries

As a first step the following necessary libraries are imported:

*   numpy to handle data in a vectorized manner
*   json to handle JSON files
*   geocoders to convert adress into latitude and longituded values
*   Matplotlib for plotting features
*   Kmeans fromn sklearn as a statistical funtction
*   folium for rendering maps

In [ ]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         235 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.4.13-~ --> conda-forge::ca-certificates-

## Get borough and neigborhood data from New York

as a second step we download a JSON file containing the required data from New York

In [ ]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


the the data is loaded to provide a first overview of the content

In [ ]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [ ]:
neighborhoods_data = newyork_data['features']


In [ ]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

then the json file format is tranformed in a pandas dataframe structe by first creating a empty dataframe and then loop through the data and fill it it to the created dataframe one row at a time

In [ ]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [ ]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Now let's have a look at the created dataframe and sorting it by borough and the nummber of neighborhoods

In [ ]:
Borough_data = neighborhoods['Borough']
Borough_data.value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

Due to the fact that we are only interested in the neighborhoods of the Bronx and Queens, we create an additional dataframe containing only the data from the neighborhoods of the Bronx and Queens with the corresponing latitude and longitude

In [ ]:
battle_data = neighborhoods[(neighborhoods['Borough'] == 'Queens') | (neighborhoods['Borough'] == 'Bronx')].reset_index(drop=True)
battle_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Then we visualize the date from the battle_data dataframe with markers on the map using folium and geolocator

In [ ]:
address = 'Queens, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of Manhattan using latitude and longitude values
map_battle = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, label in zip(battle_data['Latitude'], battle_data['Longitude'], battle_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_battle)  
    
map_battle

## Call Foursquare API funtionalites

### Now we are going to start utilizing the Foursquare APE to explore the neighborhoods and segment them

as a first sept we define the Foursquare crendential and the version

In [ ]:
CLIENT_ID = 'R2DQ22O3QSECL23QFAUCH4T4Z33PVKEL22KUFK0OMOBP5EAO' # your Foursquare ID
CLIENT_SECRET = 'TQMPOKCYAAHQR35C05OX5WIQI20HTR3UW2QGM2IGBX3ISHUM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R2DQ22O3QSECL23QFAUCH4T4Z33PVKEL22KUFK0OMOBP5EAO
CLIENT_SECRET:TQMPOKCYAAHQR35C05OX5WIQI20HTR3UW2QGM2IGBX3ISHUM


As a second step we create a function to get the venues within a radius of 1000 meters for every neighborhood located either in the Bronx or in Queens:


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The we write a code to run the above function on each neighborhood and create a nwe dataframm called battel_venues:

In [ ]:
battle_venues = getNearbyVenues(names=battle_data['Neighborhood'],
                                   latitudes=battle_data['Latitude'],
                                   longitudes=battle_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cam

Let's check the size of the resulting dataframe battle_venues

In [ ]:
print(battle_venues.shape)
battle_venues.head()

(8569, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop


Now we are interested in the amount of venues returned for eacfh neighborhood:

In [ ]:
battle_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,62,62,62,62,62,62
Arverne,37,37,37,37,37,37
Astoria,100,100,100,100,100,100
Astoria Heights,80,80,80,80,80,80
Auburndale,100,100,100,100,100,100
Bay Terrace,63,63,63,63,63,63
Baychester,100,100,100,100,100,100
Bayside,100,100,100,100,100,100
Bayswater,10,10,10,10,10,10


Finally, we would like to state the unique categories within the dataframe

In [ ]:
print('There are {} uniques categories.'.format(len(battle_venues['Venue Category'].unique())))

There are 378 uniques categories.


# Analyzing each neighborhood according to the provided categories
As a first step we are going to add back the neighborhood column to the dataframe and move it to the first column

In [ ]:
battle_onehot = pd.get_dummies(battle_venues[['Venue Category']], prefix="", prefix_sep="")
battle_onehot['Neighborhood'] = battle_venues['Neighborhood'] 
fixed_columns = [battle_onehot.columns[-1]] + list(battle_onehot.columns[:-1])
battle_onehot = battle_onehot[fixed_columns]
battle_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Adult Education Center,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Gym,College Residence Hall,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Puerto Rican Restaurant,Ramen Restaurant,Record Shop,Recor

Now, let's have a look on the size of the new created battle_onehot dataframe

In [ ]:
battle_onehot.shape

(8569, 378)

Then we would like to figure out the frequency of occurence of each category in the respective neighborhoods within the dataframe battle_onehot by defining a new dataframe calling battle_grouped

In [ ]:
battle_grouped = battle_onehot.groupby('Neighborhood').mean().reset_index()
battle_grouped

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Adult Education Center,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Gym,College Residence Hall,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Puerto Rican Restaurant,Ramen Restaurant,Record Shop,Recor

Let's have look on the new size of the new created dataframe

In [ ]:
battle_grouped.shape

(133, 378)

Now we would like to have an overview of the top 3 categories in regards of the cummulated frequence of occurence within each neighborhood


In [ ]:
num_top_venues = 3

for hood in battle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = battle_grouped[battle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
           venue  freq
0    Pizza Place  0.08
1  Deli / Bodega  0.06
2     Donut Shop  0.06


----Arverne----
        venue  freq
0   Surf Spot  0.11
1       Beach  0.11
2  Donut Shop  0.05


----Astoria----
              venue  freq
0               Bar  0.08
1  Greek Restaurant  0.05
2       Pizza Place  0.05


----Astoria Heights----
                 venue  freq
0  Rental Car Location  0.08
1        Deli / Bodega  0.05
2          Bus Station  0.05


----Auburndale----
               venue  freq
0  Korean Restaurant  0.12
1        Pizza Place  0.05
2   Greek Restaurant  0.04


----Bay Terrace----
            venue  freq
0  Clothing Store  0.06
1   Women's Store  0.05
2            Bank  0.05


----Baychester----
            venue  freq
0  Clothing Store  0.05
1      Donut Shop  0.04
2        Pharmacy  0.04


----Bayside----
              venue  freq
0               Bar  0.05
1       Pizza Place  0.05
2  Sushi Restaurant  0.04


----Bayswater----
           venue  freq


## let's put those result into a pandas dataframe for further analysis
First, we write a function to sort the venues in descending order 

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then we create a new dataframe to display the top 3 venues of each neibhorhood

In [ ]:
num_top_venues = 3
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = battle_grouped['Neighborhood']
for ind in np.arange(battle_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(battle_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted1.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Donut Shop
1,Arverne,Beach,Surf Spot,Deli / Bodega
2,Astoria,Bar,Greek Restaurant,Pizza Place
3,Astoria Heights,Rental Car Location,Deli / Bodega,Italian Restaurant
4,Auburndale,Korean Restaurant,Pizza Place,Greek Restaurant


## Cluster neighborhoods by using k-means clustering

Setting up the k-meains function by definining the number of cluster = 3, run it and check the cluster labels for each row in the dataframe

In [ ]:
kclusters = 3
battle_grouped_clustering = battle_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(battle_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 0, 2, 2, 2, 2, 2, 2, 1, 2], dtype=int32)

The create a new dataframe that inclueds the clsuter as well as the top 3 venues of each neighborhood

In [ ]:
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans.labels_)
battle_merged = battle_data
battle_merged = battle_merged.join(neighborhoods_venues_sorted1.set_index('Neighborhood'), on='Neighborhood')
battle_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1,Pharmacy,Supermarket,Caribbean Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,2,Department Store,Mobile Phone Shop,Shopping Mall
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Deli / Bodega,Diner
3,Bronx,Fieldston,40.895437,-73.905643,2,Bar,Pizza Place,Bus Station
4,Bronx,Riverdale,40.890834,-73.912585,2,Park,Pizza Place,Bar


##  Visualize the results on a map and furhter examine the clusters

Let's have a look on the clusters on the map by setting up the map, define colour schemes for the cluster markers and by adding the markers on the map



In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(battle_merged['Latitude'], battle_merged['Longitude'], battle_merged['Neighborhood'], battle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

As a final step we examine each of the 3 clusters to determine the venue categories that destinguish each cluster


## Cluster 0

In [ ]:
battle_merged.loc[battle_merged['Cluster Labels'] == 0, battle_merged.columns[[1] + list(range(5, battle_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
87,Broad Channel,Other Nightlife,Park,Playground
88,Breezy Point,Surf Spot,Pizza Place,Trail
93,Arverne,Beach,Surf Spot,Deli / Bodega
94,Rockaway Beach,Beach,Ice Cream Shop,Bar
95,Neponsit,Beach,Deli / Bodega,Boutique
106,Belle Harbor,Beach,Boutique,Spa
107,Rockaway Park,Beach,Pizza Place,Bar
124,Roxbury,Art Gallery,Theater,Beach
130,Hammels,Beach,Surf Spot,Donut Shop


## Cluster 1

In [ ]:
battle_merged.loc[battle_merged['Cluster Labels'] == 1, battle_merged.columns[[1] + list(range(5, battle_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Wakefield,Pharmacy,Supermarket,Caribbean Restaurant
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner
7,Norwood,Pizza Place,Sandwich Place,Donut Shop
8,Williamsbridge,Caribbean Restaurant,Pizza Place,Fried Chicken Joint
13,University Heights,Grocery Store,Pizza Place,Fried Chicken Joint
14,Morris Heights,Food Truck,Pharmacy,Deli / Bodega
19,Melrose,Mexican Restaurant,Donut Shop,Grocery Store
22,Longwood,Fast Food Restaurant,Grocery Store,Mexican Restaurant
24,Morrisania,Fast Food Restaurant,Grocery Store,Discount Store
25,Soundview,Fried Chicken Joint,Department Store,Fast Food Restaurant


## Cluster 2

In [ ]:
battle_merged.loc[battle_merged['Cluster Labels'] == 2, battle_merged.columns[[1] + list(range(5, battle_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Co-op City,Department Store,Mobile Phone Shop,Shopping Mall
3,Fieldston,Bar,Pizza Place,Bus Station
4,Riverdale,Park,Pizza Place,Bar
5,Kingsbridge,Pizza Place,Mexican Restaurant,Sandwich Place
6,Woodlawn,Deli / Bodega,Pizza Place,Pub
9,Baychester,Clothing Store,Pizza Place,Department Store
10,Pelham Parkway,Pizza Place,Bus Station,Deli / Bodega
11,City Island,Harbor / Marina,Seafood Restaurant,Italian Restaurant
12,Bedford Park,Pizza Place,Park,Diner
15,Fordham,Italian Restaurant,Pizza Place,Spanish Restaurant
